# 02 - Filtering and Sorting Data



### Step 1. Import the necessary libraries

In [2]:
import json         # Trabaja con ficheros json
import pandas as pd # Librería para el data mining y data wrangling
import numpy as np

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). Assign it to a variable called chipo.

This time we are going to pull data directly from the internet.

In [3]:
address = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
df_chipotle = pd.read_csv(address, sep='\t')
df_chipotle.info()
df_chipotle

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


### Step 3. Name of the most expensive products (item_name)

¿Recuerdas el `map`? En **pandas** tienes el método [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) y en para resolver este ejercicio te puede ser muy útil. 

Pero no acaba ahí https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas

In [53]:
# Minifunción: elimina el símbolo de dolar y convierte la variable en float
minif = lambda x: float(x.replace('$',''))
# Se aplica la minifunción a toda la tabla para poder hacer cálculos
df_chipotle['price_no$'] = df_chipotle.item_price.apply(minif)
# se calcula el precio por artículo (item_price/quantity) y se genera una nueva columna 'real item price'
df_chipotle['real_item_price'] = df_chipotle['price_no$']/df_chipotle.quantity

MostExpensive = df_chipotle['item_name'][df_chipotle['real_item_price'].idxmax()]
Pricest = df_chipotle.real_item_price.max()
print(f"The most expensive product is the {MostExpensive}, which costs ${Pricest}.")

The most expensive product is the Steak Salad Bowl, which costs $11.89.


### Step 4. How many products cost more than $10.00?

In [56]:
df_chipotle_10 = df_chipotle.copy()
# Lo primero es filtrar los elementos que NO se desean para aligerar las consultas
price10 = df_chipotle_10['real_item_price'] >= 10
df_chipotle_10 = df_chipotle_10[price10]

# Se compone el concepto de producto: item+toppings
df_chipotle_10['product'] = df_chipotle_10['item_name'] + '- ' + df_chipotle_10['choice_description']

# Se eliminan los elementos que ya no se necesitan
df_chipotle_10.pop('item_name')
df_chipotle_10.pop('choice_description')
df_chipotle_10.pop('order_id')

# Se genera una nueva tabla con las columnas deseadas 
df_chipotle_10 = df_chipotle_10[['product','real_item_price']]
df_chipotle_10 = df_chipotle_10.rename({'product':'Product','real_item_price':'Price'}, inplace = False)
df_chipotle_10.groupby('product').sum()
df_chipotle_10.drop_duplicates()
df_chipotle_10.to_csv('ProductPrice10+.csv', sep='|', index=False)
df_chipotle_10

"""
No consigo eliminar los duplicados de la lista
será por los ínidices??
"""

,product,real_item_price
5,"Chicken Bowl- [Fresh Tomato Salsa (Mild), [Ric...",10.98
7,"Steak Burrito- [Tomatillo Red Chili Salsa, [Fa...",11.75
13,"Chicken Bowl- [Fresh Tomato Salsa, [Fajita Veg...",11.25
23,Chicken Burrito- [[Tomatillo-Green Chili Salsa...,10.98
39,"Barbacoa Bowl- [Roasted Chili Corn Salsa, [Faj...",11.75
...,...,...
4610,"Steak Burrito- [Fresh Tomato Salsa, [Rice, Sou...",11.75
4611,"Veggie Burrito- [Tomatillo Green Chili Salsa, ...",11.25
4617,"Steak Burrito- [Fresh Tomato Salsa, [Rice, Bla...",11.75
4618,"Steak Burrito- [Fresh Tomato Salsa, [Rice, Sou...",11.75


### Step 5. What is the price of each item? 
###### print a data frame with only two columns item_name and item_price

In [43]:
df_chipotle_item = df_chipotle.copy()
#df_chipotle_item.drop(['order_id','quantity','item_name','item_price','price_no$'], axis= 1)
df_chipotle_item.pop('order_id')
df_chipotle_item.pop('price_no$')
df_chipotle_item.pop('quantity')
df_chipotle_item.pop('choice_description')
df_chipotle_item.pop('real_item_price')
df_chipotle_item

,item_name,item_price
0,Chips and Fresh Tomato Salsa,$2.39
1,Izze,$3.39
2,Nantucket Nectar,$3.39
3,Chips and Tomatillo-Green Chili Salsa,$2.39
4,Chicken Bowl,$16.98
...,...,...
4617,Steak Burrito,$11.75
4618,Steak Burrito,$11.75
4619,Chicken Salad Bowl,$11.25
4620,Chicken Salad Bowl,$8.75


### Step 6. Sort by the name of the item

In [44]:
df_chipotle_item.sort_values(by=['item_name'])

,item_name,item_price
3389,6 Pack Soft Drink,$12.98
341,6 Pack Soft Drink,$6.49
1849,6 Pack Soft Drink,$6.49
1860,6 Pack Soft Drink,$6.49
2713,6 Pack Soft Drink,$6.49
...,...,...
2384,Veggie Soft Tacos,$8.75
781,Veggie Soft Tacos,$8.75
2851,Veggie Soft Tacos,$8.49
1699,Veggie Soft Tacos,$11.25


### Step 7. What was the quantity of the most expensive item ordered? 2 ways

V1

In [67]:
maxItemPrice = lambda x: x >= df_chipotle['price_no$'].max()
df_chipotle_itemV1 = df_chipotle.copy()
MaxPrice = df_chipotle_itemV1['price_no$'].apply(maxItemPrice)
df_chipotle_itemV1 = df_chipotle_itemV1[MaxPrice]
df_chipotle_itemV1

""" EL SIGUIENTE CÓDIGO NO SALE 'LIMPIO' ASÍ QUE NO LO USO
quantity = df_chipotle_itemV1['quantity']
it_price = df_chipotle_itemV1['item_price']
it_name = df_chipotle_itemV1['item_name']
print(f'The quantity of the most expensive item ordered ({it_name}, for a total of {it_price}), was {quantity} units')
"""

,order_id,quantity,item_name,choice_description,item_price,price_no$,real_item_price
3598,1443,15,Chips and Fresh Tomato Salsa,NaN,$44.25,44.25,2.95


V2

In [74]:
df_chipotle_itemV2 = df_chipotle.copy()
#df_chipotle_item.drop(['order_id','quantity','item_name','item_price','price_no$'], axis= 1)
df_chipotle_itemV2.pop('order_id')
df_chipotle_itemV2.pop('choice_description')
df_chipotle_itemV2.pop('real_item_price')
df_chipotle_itemV2.sort_values(by=['price_no$'], ascending=False).head(1)

,quantity,item_name,item_price,price_no$
3598,15,Chips and Fresh Tomato Salsa,$44.25,44.25


### Step 8. How many times was a Veggie Salad Bowl ordered?

In [102]:
df_chipotle_Vegg = df_chipotle.copy()

# Eliminar columnas inútiles
df_chipotle_Vegg.pop('order_id')
df_chipotle_Vegg.pop('choice_description')
df_chipotle_Vegg.pop('real_item_price')
df_chipotle_Vegg.pop('item_price')
df_chipotle_Vegg.pop('price_no$')

# Eliminar filas inútiles
Vegg_Bowl = df_chipotle_Vegg['item_name'] == 'Veggie Salad Bowl'
df_chipotle_Vegg = df_chipotle_Vegg[Vegg_Bowl]

# Agrupar datos
df_chipotle_Vegg = df_chipotle_Vegg.groupby(['item_name']).sum()
df_chipotle_Vegg

,quantity
item_name,
Veggie Salad Bowl,18


### Step 9. How many times did someone order more than one Canned Soda?

In [104]:
df_chipotle_Orders = df_chipotle.copy()

# Eliminar columnas inútiles
df_chipotle_Orders.pop('order_id')
df_chipotle_Orders.pop('choice_description')
df_chipotle_Orders.pop('real_item_price')
df_chipotle_Orders.pop('item_price')
df_chipotle_Orders.pop('price_no$')

# Eliminar filas inútiles
# Primero las que no son 'Canned Soda'
canned_soda = df_chipotle_Orders['item_name'] == 'Canned Soda'
df_chipotle_Orders = df_chipotle_Orders[canned_soda]

# Segundo las que no pidieron mas de un 'Canned Soda'
quan_1 = df_chipotle_Orders['quantity'] > 1
df_chipotle_Orders = df_chipotle_Orders[quan_1]
df_chipotle_Orders.count()



quantity     20
item_name    20
dtype: int64